In [ ]:
# default_exp b1259
!date

Mon May 31 09:41:04 PDT 2021


# B1259-63 Analysis
> Create a B1259-63 (aka PSR J1302-6350) light curve using Bayesian Blocks  

* [HESS high-energy results](https://arxiv.org/pdf/astro-ph/0506280.pdf)

* [Previouis *fermi* paper](https://arxiv.org/pdf/1912.05868.pdf)

In [ ]:
# export
import numpy as np
import matplotlib.pyplot as plt
from wtlike  import *

from utilities.ipynb_docgen import *

## Special data check

In [ ]:
#hide
from wtlike.data_man import check_data, update_recent
#check_data()
#clear = update_recent()
clear=False
#update_recent(); clear=True

In [ ]:
# export

class B1259Periastron(WtLike):
    """
    """    
    tp, period = 55544.694, 1236.7243
    
    def __init__(self, **kwargs):
        super().__init__('PSR B1259-63',**kwargs)

    @property
    def mjd_dates(self):
        return [self.tp+n*self.period for n in range(4)]
         
    def date_info(self):
        return pd.DataFrame([dict(MJD=round(d,3), UTC=UTC(d)) for d in self.mjd_dates ])
    
    def stacked_plots(self, fignum=3, ylim=(2,200), ts_min=4, p0=0.5):
        dailies = [self.view(tz, tz+125,1) for tz in self.mjd_dates]
        self.bb_views = [daily.bb_view(p0=p0) for daily in dailies]
        
        fignum=1
        fig, axx = plt.subplots(4,1, sharex=True, sharey=True, figsize=(12,10), num=fignum)
        plt.subplots_adjust(hspace=0, top=0.92)
        for i, (bbv, ax, tzero) in enumerate(zip(self.bb_views, axx.flatten(), self.mjd_dates)):
            bbv.plot(ax=ax, show_flux=True, tzero =tzero, xlim=(0, 125),
                                    source_name=f'{UTC(tzero)[:4]}', ylabel='',
                        legend_loc='upper left' if i==0 else 'none',
                       error_pixsize=0);
        ax.set(xlabel='Days past periastron')
        fig.suptitle('PSR B1259-63 peristron behavior')
        fig.text(0.04, 0.5,'Count flux ($\mathrm{10^{-6}\ cm^{-2}\ s^{-1}}$)', rotation='vertical', va='center' );

        fig.width=600
        return fig

In [ ]:
weekly = self=B1259Periastron(clear= clear)

SourceData: photons and exposure for PSR B1259-63: Restoring from cache with key "PSR_B1259-63_data"
SourceData: Source PSR B1259-63 with:
	 data:       766,105 photons from 2008-08-04 to 2021-05-31
	 exposure: 3,365,830 intervals,  average flux 3478 cm^2 for 100.5 Ms
	 rates:  source 7.88e-09/s, background 2.18e-06/s, S/N ratio 3.61e-03
CellData: Bin photon data into 668 1-week bins from 54683.0 to 59359.0
LightCurve: select 661 cells for fitting with e>1 & n>2


In [ ]:
#collapse_hide
def B1259( clear=False):
    r"""
    ## Fit to all data

    {date}
    
    Create a `WtLike` object with all the data
    
    {outp}
 
    ## The full weekly-interval light curve, showing the BB partitions
    {out2}
    {fig2}
    
    ## Expand about each periastron
 
    #### Periastron dates

    Assuming {period}-day orbital period, the MJD and UTC values are:
    
    {utc}
 
    Expand the above about those dates

    {fig3}
    
    ## Recent detail 
    {out4}{fig4}
    
    """
    global weekly #  make availlable for follow-up cells

 
    pd.set_option('display.precision', 4)#, 'display.colheader_justify','left')

    
    with capture_print('Output from analysis: create cells, fit each, run BB partition, fit partitions.') as outp:
        if weekly is None or clear:
            weekly = B1259Periastron(time_bins=(0,0,7), clear=clear)#.bb_view()
        else:
            print('(already done)')
        bb = weekly.bb_view(0.5)
    
    df_text = monospace(str(bb.fluxes), 'BB fit table', open=True)

    # fig 2: full light curbe
    plt.rc('font', size=16)
    
    fig2 = figure( bb.plot(fignum=2, figsize=(15,4)),
                  width=600)
    
    # fig 3 -- stack light curves
    utc = monospace(str(weekly.date_info())) 
    with capture_print('Output from periastron analyses') as out2:
        fig3 = figure( weekly.stacked_plots( ylim=(4,300), fignum=3), width=600)
    
    # fit 4 -- last two weeks
    with capture_print(f'Output from last two week analysis') as out4:
        recent_wk = weekly.view(-14,0,1)
        print(recent_wk.fluxes)
        fig4=figure(
            recent_wk.plot(show_flux=True, ylim=(-.1,2), title='Last 2 weeks',
                       xlabel=f'MJD, until {UTC(weekly.stop)}',fignum=4),
            caption=None, width=600)
  
    return locals()

nbdoc(B1259, clear=False)

## Fit to all data

2021-05-31 09:41

Create a `WtLike` object with all the data

<details  class="nbdoc-description" >  <summary> Output from analysis: create cells, fit each, run BB partition, fit partitions. </summary>  <div style="margin-left: 5%"><pre>(already done)<br>LightCurve: select 661 cells for fitting with e&gt;1 & n&gt;2<br>Bayesian Blocks: using penalty 0.5<br>Partitioned 661 cells into 30 blocks, using LikelihoodFitness <br>LightCurve: Loaded 30 / 30 cells for fitting<br></pre></div> </details>

## The full weekly-interval light curve, showing the BB partitions
<details  class="nbdoc-description" >  <summary> Output from periastron analyses </summary>  <div style="margin-left: 5%"><pre>CellData: Bin photon data into 125 1-day bins from 55544.7 to 55669.7<br>LightCurve: select 119 cells for fitting with e&gt;1 & n&gt;2<br>CellData: Bin photon data into 125 1-day bins from 56781.4 to 56906.4<br>LightCurve: select 125 cells for fitting with e&gt;1 & n&gt;2<br>CellData: Bin photon data into 125 1-day bins from 58018.1 to 58143.1<br>LightCurve: select 125 cells for fitting with e&gt;1 & n&gt;2<br>CellData: Bin photon data into 125 1-day bins from 59254.9 to 59379.9<br>LightCurve: select 111 cells for fitting with e&gt;1 & n&gt;2<br>LightCurve: select 119 cells for fitting with e&gt;1 & n&gt;2<br>Bayesian Blocks: using penalty 0.5<br>Partitioned 119 cells into 12 blocks, using LikelihoodFitness <br>LightCurve: Loaded 12 / 12 cells for fitting<br>LightCurve: select 125 cells for fitting with e&gt;1 & n&gt;2<br>Bayesian Blocks: using penalty 0.5<br>Partitioned 125 cells into 20 blocks, using LikelihoodFitness <br>LightCurve: Loaded 20 / 20 cells for fitting<br>LightCurve: select 125 cells for fitting with e&gt;1 & n&gt;2<br>Bayesian Blocks: using penalty 0.5<br>Partitioned 125 cells into 16 blocks, using LikelihoodFitness <br>LightCurve: Loaded 16 / 16 cells for fitting<br>LightCurve: select 111 cells for fitting with e&gt;1 & n&gt;2<br>Bayesian Blocks: using penalty 0.5<br>Partitioned 111 cells into 16 blocks, using LikelihoodFitness <br>LightCurve: Loaded 16 / 16 cells for fitting<br></pre></div> </details>
<figure style="margin-left: 5%" title="Figure 1">  <a href="images/B1259_fig_01.png" title="images/B1259_fig_01.png">    <img src="images/B1259_fig_01.png" alt="Figure 1 at images/B1259_fig_01.png" width=600>   </a> </figure>

## Expand about each periastron

#### Periastron dates

Assuming {period}-day orbital period, the MJD and UTC values are:

<div style="margin-left: 5%"><pre>         MJD               UTC<br>0  55544.694  2010-12-14 16:39<br>1  56781.418  2014-05-04 10:02<br>2  58018.143  2017-09-22 03:25<br>3  59254.867  2021-02-09 20:48</pre></div>

Expand the above about those dates

<figure style="margin-left: 5%" title="Figure 2">  <a href="images/B1259_fig_02.png" title="images/B1259_fig_02.png">    <img src="images/B1259_fig_02.png" alt="Figure 2 at images/B1259_fig_02.png" width=600>   </a> </figure>

## Recent detail 
<details  class="nbdoc-description" >  <summary> Output from last two week analysis </summary>  <div style="margin-left: 5%"><pre>CellData: Bin photon data into 14 1-day bins from 59351.0 to 59365.0<br>LightCurve: select 14 cells for fitting with e&gt;1 & n&gt;2<br>          t   tw    n    ts      flux             errors     limit<br>0   59351.5  1.0  168  27.4   88.9267  (-20.613, 22.308)  128.3873<br>1   59352.5  1.0  175  15.8   60.4052   (-17.942, 19.68)   95.6443<br>2   59353.5  1.0   19   0.1    7.9583   (-7.958, 46.069)  112.7259<br>3   59354.5  1.0  182  18.3   75.4740  (-20.623, 22.445)  115.3825<br>4   59355.5  1.0  207  36.6  117.3467  (-24.109, 25.252)  160.7021<br>5   59356.5  1.0  165  16.5   72.8223  (-21.122, 23.097)  114.0655<br>6   59357.5  1.0  180  29.4   96.5105  (-21.944, 23.743)  138.4994<br>7   59358.5  1.0  143   3.3   34.2553  (-19.682, 21.706)   73.6425<br>8   59359.5  1.0  152   9.2   43.3002   (-16.606, 18.91)   78.3277<br>9   59360.5  1.0   42   7.5  114.9164  (-48.631, 56.903)  222.9075<br>10  59361.5  1.0  124   5.8   39.0425  (-17.657, 19.722)   75.0019<br>11  59362.5  1.0  146  10.2   52.6856  (-18.598, 20.551)   89.7379<br>12  59363.5  1.0  144   1.0   15.8136  (-15.657, 18.009)   50.7971<br>13  59364.5  1.0  150   2.5   22.2998  (-14.809, 17.016)   54.4844<br></pre></div> </details><figure style="margin-left: 5%" title="Figure 3">  <a href="images/B1259_fig_03.png" title="images/B1259_fig_03.png">    <img src="images/B1259_fig_03.png" alt="Figure 3 at images/B1259_fig_03.png" width=600>   </a> </figure>


<!-- ## Questions for Discussion, with 2/28 Scargle comments

#### How do I interpret the apparent abrupt transitions? 
>Think of BB as the best step-function representation
of the data.  It does not mean that the discontinuities 
are claimed to be real -- the underlying light curve
is most likely continuous and smooth at some level.  
BB is just saying that there is a statistically significant
change from one block to the next.

Compare with a [flare from 3C 279](https://tburnett.github.io/wtlike/bayesian.html#bb_overplot), which has measurements rather than limits for each day and the time scale is in the day range. 

#### Is the day interval appropriate? 
>BB can always benefit from finer sampling, so if you
have data on shorter intervals (or even photon data)
you can get more information out ... e.g. a better 
determination of the time of "transitions" (see above).



#### I know that there is a ~4% systematic for the individual daily measurements, seen in the Geminga data. Does this matter?
>Systematic errors are a problem ... BB is not magic here.

## My observations, plans

There are two related questions here. 
1. what is the actual behavior of B1259, besides the obvious flares? Can we characterize each periastron to detect changes? Kent Wood is very interested in this.
2. How well does BB work, when applied to daily likelihoods, many of which are just limits, but do contain information? This is a question that Jeff Scargle had never considered before, but is quite interested in.


#### Run Simulations

I'll use the [wtlike simulation capability](https://tburnett.github.io/wtlike/simulation) to test the sensitivity to various light curves, when daily fits are all or mostly limits.

#### Add Systematics to Likelihood
The presence of a systematic error in the exposure--I've measured 4% with Geminga, slightly invalidates the likelihood. A fix is to widen it, which I can easily do using the [wtlike Poisson representation](https://tburnett.github.io/wtlike/poisson#Poisson).  -->

In [ ]:
#hide
# recent_wk = weekly.view(-14,0,1)
# print(recent_wk.fluxes)
# recent_wk.plot(show_flux=True, ylim=(-.1,2), title='Last 2 weeks',
#                xlabel=f'MJD, until {UTC(weekly.stop)}',);
# #recent_wk.bb_view(0.5).fluxes; ff

# recent_wk.bb_view(0.5).fluxes

# recent_wk.bb_view(0.5).fluxes.flux*weekly.src_flux*1e6

# all_wks = weekly.view(0,0,1)

# all_wks.fluxes.hist('ts', bins=np.linspace(0,50, 51), log=True);

In [ ]:
#hide
# Comparison with Tyrel's gtlike results

# hide
#collapse_input
# remote_file = '/nfs/farm/g/glast/u/tyrelj/forToby/FluxLC_output_PSRB1259.fits'
# local_copy = '/home/burnett/fromTyrel/FluxLC_output_PSRB1259.fits'
# from astropy.io import fits

# with fits.open(local_copy) as hdus: 
#     tdata = hdus[1].data

# flux_factor = 4.28e7 # derived before, so apply here
# tdf = pd.DataFrame(dict(t=MJD((tdata.Start+tdata.Stop)/2).round(1),
#                         TS = tdata.TS.astype(float).round(1),
#                         flux = (tdata.flux.astype(float)*flux_factor).round(1),
#                         flux_error = (tdata.flux_error.astype(float)*flux_factor).round(1),
#                         limit = (tdata.UpperLimit.astype(float)*flux_factor).round(1),
#                        )
#                     ); 
#tdf.describe(percentiles=[])

# #### Get my results for these dates


# lc=daily.lc; 
# mdf = lc.dataframe.query('59319>t>59093').copy()
# mdf.loc[:,'ts']= mdf.fit.apply(lambda x: x.ts)
# mdf.loc[:,'flux']=mdf.fit.apply(lambda x: round(x.flux,1) )
# mdf.loc[:,'errors']= mdf.fit.apply(lambda x: (np.array(x.errors)-x.flux).round(1))
# mdf.loc[:,'limit']= mdf.fit.apply(lambda x: x.limit.round(1))
# #mdf['t ts flux errors limit'.split()].describe(percentiles=[])

# ### Use the last one, at 59318.5, +63, to make tentative normalization


# f = mdf.iloc[-1].fit
# print(f'Compare point at {mdf.iloc[-1].t}: TS: me {f.ts:.1f}, Tyrel {tdf.iloc[-1].TS:.1f}')
# a,b = f.flux, tdf.iloc[-1].flux; a,b, 
# print(f'flux ratio check: {a/b:.3e}')

# fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(15,5))
# ax1.plot(mdf.ts, tdf.TS, 'o');
# ax1.set(xlabel='wtlike TS', ylabel='gtlike TS')
# ax1.grid(alpha=0.5)
# mtscut= (mdf.ts>4).values
# ax2.plot(mdf.flux, tdf.flux, 'o');
# ax2.plot(mdf.flux[mtscut], tdf[mtscut].flux, 'or', label='wtlike TS>4');
# ax2.plot([0,70], [0,70], '--', color='grey');
# ax2.set(xlabel='wtlike flux', ylabel=f'gtlike flux x {flux_factor:.2e}', xlim=(0,70), ylim=(0,70))
# ax2.legend();ax2.grid(alpha=0.5);

# ttscut=(tdf.TS>4).values
# ax3.plot(mdf.flux, tdf.flux, 'o');
# ax3.plot(mdf.flux[ttscut], tdf[ttscut].flux, 'or', label='gtlike TS>4');
# ax3.plot([0,70], [0,70], '--', color='grey');
# ax3.legend()
# ax3.set(xlabel='wtlike flux', ylabel=f'gtlike flux x {flux_factor:.2e}', xlim=(0,70), ylim=(0,70))
# ax3.legend();ax3.grid(alpha=0.5);

In [ ]:
#hide
### Add my TS, flux to Tyrel list

# %nbdev_collapse_input
# tdf.loc[:,'wt_ts']= mdf.ts.values
# tdf.loc[:,'wt_flux']=mdf.flux.values
# tdf.loc[:,'wt_errors'] = mdf.errors.values

# pa = daily; tzero = pa.date_info().MJD[3]
# fig, ax = plt.subplots(figsize=(15,4))
# mcut=mdf.ts.values>4
# tcut=tdf.TS.values>4
# ax.plot((mdf.t-tzero)[mcut], mdf.flux[mcut], 'x',ms=10, label='wtlike')
# ax.plot((tdf.t-tzero)[tcut], tdf.flux[tcut], '+', ms=15, label='gtlike')
# ax.set(yscale='linear', ylim=(1,90), xlim=(-60, 70), xlabel='Days about periastron', 
#        ylabel='Relative Flux');
# # ax.legend(loc='upper center', title='TS>4 cut on respective data sets')
# ax.grid(alpha=0.5);

In [ ]:
#hide
# #### Generate table of 30-day fits, to compare with LightCurveRepository

# wt3 = weekly.view((0., 0, 30))
# wt3.plot();

# ft = wt3.flux_table()
# ft.to_csv('../b1259_3day.csv')

# ft